### 1. 기본 얼굴 검출

In [7]:
import cv2

In [8]:
cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'

# Haar Cascade 로드
face_cascade = cv2.CascadeClassifier(cascade_path)

In [9]:
# 웹캠 열기
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("웹캠이 연결이 안됩니다")

while True:

    # 프레임 읽기
    ret, frame = cap.read()

    if not ret:
        break

    # 그레이스케일 - 흑백 변환
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 얼굴 검출
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30,30))

    for i in faces:
        x, y, w, h = i
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

    # 출력
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [10]:
# 넣을 이모티콘 이미지 로드
emoji_img = cv2.imread('cat_face.png')
cv2.imshow("emoji_img", emoji_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
# 웹캠 열기
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("웹캠이 연결이 안됩니다")

while True:

    # 프레임 읽기
    ret, frame = cap.read()

    if not ret:
        break

    # 그레이스케일 - 흑백 변환
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 얼굴 검출
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30,30))

    for i in faces:
        x, y, w, h = i
        # cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        emoji_img_resized = cv2.resize(emoji_img, (w, h))
        
        frame[y:y+h, x:x+w] = emoji_img_resized

    # 출력
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### 3. 자연스럽게 다양한 효과를 적용해서 합성해보기

In [12]:
# 넣을 이모티콘 이미지 로드
emoji_img = cv2.imread('cat_face.png', cv2.IMREAD_UNCHANGED)
print(emoji_img.shape)
cv2.imshow("emoji_img", emoji_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

(500, 500, 4)


In [13]:
import numpy as np

emoji_alpha = emoji_img[:, :, 3] / 255.0
emoji_alpha[100:200, 200:300]

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [14]:
emoji_bgr = emoji_img[:, :, :3] # brg 채널값
emoji_bgr[100:200, 200:300]

array([[[ 23,  84, 228],
        [ 24,  85, 229],
        [ 20,  84, 226],
        ...,
        [ 35, 125, 244],
        [ 36, 122, 244],
        [ 35, 121, 245]],

       [[ 20,  85, 230],
        [ 20,  84, 232],
        [ 19,  83, 231],
        ...,
        [ 37, 136, 252],
        [ 36, 132, 250],
        [ 33, 126, 247]],

       [[ 16,  83, 228],
        [ 18,  84, 233],
        [ 18,  83, 235],
        ...,
        [ 18, 121, 237],
        [ 18, 118, 236],
        [ 17, 117, 236]],

       ...,

       [[ 35, 149, 250],
        [ 35, 149, 249],
        [ 35, 149, 249],
        ...,
        [ 31, 136, 247],
        [ 31, 133, 245],
        [ 30, 132, 244]],

       [[ 36, 147, 251],
        [ 34, 148, 249],
        [ 34, 148, 249],
        ...,
        [ 30, 132, 244],
        [ 29, 131, 244],
        [ 30, 129, 243]],

       [[ 35, 146, 250],
        [ 35, 146, 250],
        [ 33, 146, 250],
        ...,
        [ 29, 131, 243],
        [ 30, 129, 243],
        [ 29, 128, 242]]

In [16]:
# 웹캠 열기
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("웹캠이 연결이 안됩니다")

while True:

    # 프레임 읽기
    ret, frame = cap.read()

    if not ret:
        break

    # 그레이스케일 - 흑백 변환
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 얼굴 검출
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30,30))

    for i in faces:
        x, y, w, h = i

        # 넣을 이미지 크기 조정
        emoji_img_resized = cv2.resize(emoji_bgr, (w, h))

        # 알파 채널도 이미지 크기 조정
        emoji_alpha_resized = cv2.resize(emoji_alpha, (w, h))

        # 얼굴 부위 추출
        face_part = frame[y:y+h, x:x+w]

        # 배경(얼굴 부위) 반대 마스크
        background_alpha = 1.0 - emoji_alpha_resized

        # 마스크 이미지의 알파 채널과 RGB 채널을 곱해서 마스크 이미지에서 알파 처리된 부분을 추출
        alpha_face = emoji_alpha_resized[..., np.newaxis] * emoji_img_resized

        # 원본 이미지의 해당 영역과 배경 알파 채널을 곱해서 배경 추출
        background_face = background_alpha[..., np.newaxis] * face_part

        # 최종 추출된 두 부분을 합쳐서 얼굴 부분을 생성
        face_part = alpha_face + background_face

    frame[y:y+h, x:x+w] = face_part
    # 출력
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()